In [1]:
%matplotlib inline
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data
from SALib.sample import latin, saltelli, fast_sampler
from SALib.sample.morris import sample
from SALib.analyze import sobol, fast, morris
from functions.stylizedfacts import *
from functions.evolutionaryalgo import *
from functions.sensitivity_analysis import *
from functions.helpers import hurst, organise_data
import matplotlib.pyplot as plt
from matplotlib import style
import quandl

In [2]:
style.use('ggplot')

# Robustness of the results

In [ ]:
param_values = fast_sampler.sample(problem, 1000)
all_parameters = []
for parameters in param_values:
    pars = {}
    for key, value in zip(problem['names'], parameters):
        pars[key] = value
    all_parameters.append(pars)


In [ ]:
fixed_parameters = {'fundamental_value': 100,
                    'horizon_max': 22,
                    'horizon_min': 2,
                    'max_order_expiration_ticks': 24,
                    'n_traders': 1981,
                    'spread_max': 0.1034423464335639,
                    'std_fundamental': 0.10754244839080045,
                    'std_noise': 0.20680836615919262,
                    'std_vol': 11,
                    'ticks': 1000,
                    'trader_sample_size': 21,
                    'w_buy_hold': 0.0}

In [ ]:
# extract J-value & MCR
model_output = simulate_params_sobol(NRUNS=1, parameter_set=all_parameters, fixed_parameters=fixed_parameters)

In [ ]:
# calc impact on outputs of interest
impact_on_no_a = fast.analyze(problem, no_autoc, print_to_console=False)
impact_on_f_t = fast.analyze(problem, f_tails, print_to_console=False)
impact_on_cv = fast.analyze(problem, clustered_vol, print_to_console=False)
impact_on_lm = fast.analyze(problem, l_memory, print_to_console=False)
impact_on_df = fast.analyze(problem, deviation_from_fundamentals, print_to_console=False)

In [ ]:
# plot 5 histograms 
#fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(12,5))
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

N = 4
ind = np.arange(N)  # the x locations for the groups
width = 0.40       # the width of the bars

ax1 = axes[0,0] 
ax2 = axes[0,1]
ax3 = axes[0,2]
ax4 = axes[1,0]
ax5 = axes[1,1]

ax1.set_title('No autocorrelation')
rects1 = ax1.bar(ind, impact_on_no_a['S1'], width, color='k')
rects2 = ax1.bar(ind + width, impact_on_no_a['ST'], width, color='w', hatch='\\')
#ax1.legend((rects1[0], rects2[0]), ('S1', 'ST'))
ax1.set_ylabel('impact')

ax2.set_title('Fat tails')
rects3 = ax2.bar(ind, impact_on_f_t['S1'], width, color='k')
rects4 = ax2.bar(ind + width, impact_on_f_t['ST'], width, color='w', hatch='\\')

ax3.set_title('Clustered volatility')
rects5 = ax3.bar(ind, impact_on_cv['S1'], width, color='k')
rects6 = ax3.bar(ind + width, impact_on_cv['ST'], width, color='w', hatch='\\')

ax4.set_title('Long memory')
rects7 = ax4.bar(ind, impact_on_lm['S1'], width, color='k')
rects8 = ax4.bar(ind + width, impact_on_lm['ST'], width, color='w', hatch='\\')
ax4.set_ylabel('impact')

ax5.set_title('Fundamental deviation')
rects9 = ax5.bar(ind, impact_on_df['S1'], width, color='k')
rects10 = ax5.bar(ind + width, impact_on_df['ST'], width, color='w', hatch='\\')
ax5.legend((rects9[0], rects10[0]), ('First-order sensitivity', 'Total sensitivity'), 
           loc='center left', bbox_to_anchor=(1.25, 0.5))

axes[1, 2].remove()

for ax in [ax1, ax2, ax3, ax4, ax5]:
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(('F', 'MM', 'NS', 'MR'))
    ax.set_ylim(0, 1.0)
    
fig.savefig('eFastExpectations.png')